In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def khatrirao_product(tensor1, tensor2):
    i0 = tf.constant(1)
    prod0 = tf.multiply(tensor1[0, :], tensor2)
    _, khprod = tf.while_loop(lambda i, m: tf.less(i, tf.shape(tensor1)[0]),
                              lambda i, m: [i+1, tf.concat([m, tf.multiply(tensor1[i, :], tensor2)], 0)],
                              loop_vars=[i0, prod0],
                              shape_invariants=[i0.get_shape(), tf.TensorShape([None, None])])
    return khprod

In [3]:
def rank3tensor_decomposition_ALS(matrix, k, alpha=0.01, nbiter=1000):
    dim0, dim1, dim2 = matrix.shape

    #X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')
    #X_shape = tf.shape(X)
    #Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]
    X = tf.placeholder(tf.float32, shape=(dim0, dim1, dim2), name='X')
    
    X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(dim0, dim1*dim2))
    X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(dim1, dim0*dim2))
    X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(dim2, dim0*dim1))
    
    A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
    B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
    C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

    costA = tf.reduce_sum(tf.abs(X0 - tf.matmul(A, tf.transpose(khatrirao_product(C, B)))))
    costB = tf.reduce_sum(tf.abs(X1 - tf.matmul(B, tf.transpose(khatrirao_product(C, A)))))
    costC = tf.reduce_sum(tf.abs(X2 - tf.matmul(C, tf.transpose(khatrirao_product(B, A)))))
    
    trainA = tf.train.GradientDescentOptimizer(alpha).minimize(costA)
    trainB = tf.train.GradientDescentOptimizer(alpha).minimize(costB)
    trainC = tf.train.GradientDescentOptimizer(alpha).minimize(costC)
    
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)

    for _ in range(nbiter):
        sess.run(trainA, feed_dict={X: matrix})
        sess.run(trainB, feed_dict={X: matrix})
        sess.run(trainC, feed_dict={X: matrix})

    return sess.run((A, B, C), feed_dict={X: matrix})

In [4]:
def rank3tensor_decomposition_ALS_closed(matrix, k, nbiter=1000):
    dim0, dim1, dim2 = matrix.shape

    #X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')
    #X_shape = tf.shape(X)
    #Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]
    X = tf.placeholder(tf.float32, shape=(dim0, dim1, dim2), name='X')
    
    X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(dim0, dim1*dim2))
    X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(dim1, dim0*dim2))
    X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(dim2, dim0*dim1))
    
    A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
    B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
    C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

    for _ in range(nbiter):
        A = tf.matmul(tf.matmul(X0, khatrirao_product(C, B)), 
                      tf.multiply(tf.matmul(tf.transpose(C), C), tf.matmul(tf.transpose(B), B)))
        B = tf.matmul(tf.matmul(X1, khatrirao_product(C, A)), 
                      tf.multiply(tf.matmul(tf.transpose(C), C), tf.matmul(tf.transpose(A), A)))
        C = tf.matmul(tf.matmul(X2, khatrirao_product(B, A)), 
                      tf.multiply(tf.matmul(tf.transpose(B), B), tf.matmul(tf.transpose(A), A)))
        
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)
    
    return sess.run((A, B, C), feed_dict={X: matrix})

# Test: Method 1

In [5]:
# x1 = np.array([[1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12]], dtype=np.float32)
# x2 = np.array([[13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24]], dtype=np.float32)
# x = np.zeros((3, 4, 2))
# x[:, :, 0] = x1
# x[:, :, 1] = x2
x1 = np.array([[1, 0], [0, 1]], dtype=np.float32)
x2 = np.array([[0, -1], [1, 0]], dtype=np.float32)
x = np.zeros((2, 2, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

In [6]:
A, B, C = rank3tensor_decomposition_ALS(x, k=2, nbiter=10000)

In [7]:
B

array([[-0.16030072, -0.97753346],
       [ 0.9448817 , -0.1307766 ]], dtype=float32)

In [8]:
mat = np.zeros((A.shape[0], B.shape[0], C.shape[0]))

In [9]:
from itertools import product

for i, j, k in product(range(A.shape[0]), range(B.shape[0]), range(C.shape[0])):
    print('{}, {}, {}'.format(i, j, k))
    mat[i, j, k] = sum(A[i, alpha]*B[j, alpha]*C[k, alpha] for alpha in range(2))

0, 0, 0
0, 0, 1
0, 1, 0
0, 1, 1
1, 0, 0
1, 0, 1
1, 1, 0
1, 1, 1


In [10]:
mat

array([[[ 0.00137881, -0.00125296],
        [-0.00927888,  0.00120364]],

       [[ 0.01683114,  0.98761301],
        [ 0.9833448 , -0.01003787]]])

In [11]:
x

array([[[ 1.,  0.],
        [ 0., -1.]],

       [[ 0.,  1.],
        [ 1.,  0.]]])

# Method 2

In [12]:
A, B, C = rank3tensor_decomposition_ALS_closed(x, k=5, nbiter=1)

In [13]:
mat = np.zeros((A.shape[0], B.shape[0], C.shape[0]))

In [14]:
from itertools import product

for i, j, k in product(range(A.shape[0]), range(B.shape[0]), range(C.shape[0])):
    mat[i, j, k] = sum(A[i, alpha]*B[j, alpha]*C[k, alpha] for alpha in range(5))

In [15]:
mat

array([[[ 0.0313062 , -0.02590907],
        [-0.08808526,  0.07289987]],

       [[-0.01869485,  0.0154709 ],
        [ 0.0525422 , -0.04348133]]])

In [16]:
x

array([[[ 1.,  0.],
        [ 0., -1.]],

       [[ 0.,  1.],
        [ 1.,  0.]]])

In [17]:
A, B, C

(array([[ 0.00901352,  0.62060016, -0.00333547,  0.36581847,  0.16333474],
        [-0.02764865, -0.38201916, -0.03696703, -0.1044168 , -0.03626882]],
       dtype=float32),
 array([[ 2.52986240e-04, -2.75214374e-01,  1.17825344e-04,
          1.20429188e-01,  1.42035941e-02],
        [-5.97719220e-04,  7.72701561e-01, -2.62133311e-04,
         -3.51158172e-01, -4.16347943e-02]], dtype=float32),
 array([[ 4.1215394e-06, -1.7308137e-01,  1.3423387e-06,  3.9485443e-02,
          2.0211630e-03],
        [-3.3875808e-06,  1.4322491e-01, -1.0928622e-06, -3.2746196e-02,
         -1.6770517e-03]], dtype=float32))